In [6]:
"""_summary_"""

import sys
from pathlib import Path

sys.path.append(str(Path(r"C:\Users\pasupuleti\Desktop\carla-drowsiness-detection\examples\test.ipynb").parent.parent.resolve()))

from src.Driver_assistance_bot.utils import load_toml, load_json
from src.Driver_assistance_bot.tools import vibrate_steering_wheel, voice_alert
from src.Driver_assistance_bot.bot import Bot, BaseBot


# Create bot instance at module load
prompt_file = Path(r"C:\Users\pasupuleti\Desktop\carla-drowsiness-detection\src\Driver_assistance_bot\configs\prompt.toml")
schema_file = Path(r"C:\Users\pasupuleti\Desktop\carla-drowsiness-detection\src\Driver_assistance_bot\configs\schema.json")

prompt_cfg = load_toml(prompt_file)
schema_cfg = load_json(schema_file)  # Uncomment if schema mode is needed

MODEL_ID = "llama3.1:8b-instruct-q8_0"

config = Bot.BotConfig(
    model_id=MODEL_ID,
    system_prompt=prompt_cfg["SYSTEM"],
    user_prompt=prompt_cfg["USER"],
    output_schema=schema_cfg,  # or schema_cfg if structured mode
    tools=[voice_alert, vibrate_steering_wheel],
    temperature=0.1,
)

bot_instance = BaseBot(config=config)


def main(telemetry: dict):
    """
    Entry point for invoking the bot with telemetry input and executing tool calls.
    """

    response = bot_instance.invoke(Bot.Input(**telemetry))

    return response



    # for tool_call in response_.tool_calls:
    #     print("Executing tool:", tool_call.name)

In [7]:

sample_input = {
    "perclos": 0.8,
    "blink_rate": 12,
    "yawn_freq": 5,
    "sdlp": 0.8,
    "steering_entropy": 0.6,
    "steering_reversal_rate": 10,
}
response_ = main(sample_input)
print("drowsiness_level:", response_.drowsiness_level)
print("reasoning:", response_.reasoning)
print("tool_calls:", response_.tool_calls)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


drowsiness_level: High
reasoning: The driver's PERCLOS is high (0.8), blink rate is low (12.0), yawn frequency is moderate (5.0), steering entropy is low (0.6), and steering reversal rate is high (10.0). These metrics indicate a high level of drowsiness.
tool_calls: [ToolCall(name='voice_alert', args={'text': 'Alert: You are experiencing high levels of drowsiness. Please pull over to a safe location and take a break.'}, description='Alert the driver via text-to-speech.'), ToolCall(name='vibrate_steering_wheel', args={'duration': 2, 'intensity': 30}, description='Vibrate the steering wheel to alert the driver.')]


In [10]:
response_.tool_calls[0].name

'voice_alert'